In [36]:
import openai
from openai import OpenAI
import os
import base64
import requests
from key import OPENAI_API_KEY
import math
import numpy as np

In [37]:
# Initialize client
api_key=OPENAI_API_KEY
client = OpenAI(api_key=api_key)

In [38]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def list_files(directory):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# Get url name
from video import extract_youtube_id
url = extract_youtube_id("https://www.youtube.com/watch?v=Gl7m0cVa37k")

# Get all the image paths
directory = './frames/' + url  # When you run video.py, all the frames located inside frames folder
image_paths = list_files(directory)

In [39]:
directory

'./frames/Gl7m0cVa37k'

In [40]:
# Get System Prompt
f = open("prompt.txt", "r")
prompt_text = f.read()

# Get User Input
user_input = "great moments"

In [ ]:
# results = []

# # Iterate through each image path
# for image_path in image_paths:
#     # Getting the base64 string
#     base64_image = encode_image(image_path)

#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"
#     }

#     payload = {
#         "model": "gpt-4o",
#         "messages": [
#             {"role": "system", "content": prompt_text},
#             {"role": "user", "content": [{"type": "text", "text": user_input}, {"type": "image_url","image_url": {"url": f"data:image/jpg;base64,{base64_image}", "detail": "low"}}]}
#             # user_input text can be removed
#         ],
#         "max_tokens": 4096
#     }

#     # Make the API request
#     response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

#     # Append the result to the list
#     results.append(response.json())

In [ ]:
# import concurrent.futures
# import base64
# import requests

# def make_api_request(image_path, api_key, prompt_text, user_input):
#     base64_image = encode_image(image_path)

#     headers = {
#         "Content-Type": "application/json",
#         "Authorization": f"Bearer {api_key}"
#     }

#     payload = {
#         "model": "gpt-4o",
#         "messages": [
#             {"role": "system", "content": prompt_text},
#             {"role": "user", "content": [
#                 {"type": "text", "text": user_input},
#                 {"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{base64_image}", "detail": "low"}}
#             ]}
#         ],
#         "max_tokens": 4096
#     }

#     response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
#     return response.json()

# def process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=10):
#     results = []
#     with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
#         future_to_image = {executor.submit(make_api_request, image_path, api_key, prompt_text, user_input): image_path for image_path in image_paths}
#         for future in concurrent.futures.as_completed(future_to_image):
#             image_path = future_to_image[future]
#             try:
#                 result = future.result()
#                 results.append(result)
#             except Exception as exc:
#                 print(f'{image_path} generated an exception: {exc}')

#     return results

# results = process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=10)
# print(results)

In [41]:
import concurrent.futures
import base64
import requests

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def make_api_request(image_paths, api_key, prompt_text, user_input):
    # Encode all images in the list
    base64_images = [encode_image(image_path) for image_path in image_paths]

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    # Prepare the images for the payload
    images_payload = [{"type": "image_url", "image_url": {"url": f"data:image/jpg;base64,{img}", "detail": "low"}} for img in base64_images]

    payload = {
        "model": "gpt-4o",
        "messages": [
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": [
                {"type": "text", "text": user_input},
                *images_payload
            ]}
        ],
        "max_tokens": 4096
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response.json()

def process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=10, frames_per_request=3):
    results = []

    # Split image_paths into chunks of frames_per_request
    image_chunks = [image_paths[i:i + frames_per_request] for i in range(0, len(image_paths), frames_per_request)]

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_image_chunk = {executor.submit(make_api_request, chunk, api_key, prompt_text, user_input): chunk for chunk in image_chunks}
        for future in concurrent.futures.as_completed(future_to_image_chunk):
            image_chunk = future_to_image_chunk[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as exc:
                print(f'{image_chunk} generated an exception: {exc}')

    return results

results = process_images_in_parallel(image_paths, api_key, prompt_text, user_input, num_threads=20, frames_per_request=3)
print(results)

[{'id': 'chatcmpl-9gUqy4FfVIq3vTqP20Mlr5ilGeEVb', 'object': 'chat.completion', 'created': 1719915404, 'model': 'gpt-4o-2024-05-13', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '**Image Description:**\n- Main Subjects: A person is standing on a wooden pier that extends into a calm lake.\n- Actions: The person is standing still and facing away from the camera, seemingly admiring the view. They are wearing a red jacket and holding an object in their left hand.\n- Context: The scene is set in a serene outdoor location, surrounded by a reflective lake and majestic mountains in the background. The sky is overcast, and the overall atmosphere is peaceful. The clear reflection of the mountains and trees in the still water suggests a tranquil and untouched natural environment.'}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1206, 'completion_tokens': 118, 'total_tokens': 1324}, 'system_fingerprint': 'fp_4008e3b719'}, {'id': 'chatcmpl-9gUr0N0rO8

In [42]:
# Result Sample
results[0]

{'id': 'chatcmpl-9gUqy4FfVIq3vTqP20Mlr5ilGeEVb',
 'object': 'chat.completion',
 'created': 1719915404,
 'model': 'gpt-4o-2024-05-13',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '**Image Description:**\n- Main Subjects: A person is standing on a wooden pier that extends into a calm lake.\n- Actions: The person is standing still and facing away from the camera, seemingly admiring the view. They are wearing a red jacket and holding an object in their left hand.\n- Context: The scene is set in a serene outdoor location, surrounded by a reflective lake and majestic mountains in the background. The sky is overcast, and the overall atmosphere is peaceful. The clear reflection of the mountains and trees in the still water suggests a tranquil and untouched natural environment.'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 1206,
  'completion_tokens': 118,
  'total_tokens': 1324},
 'system_fingerprint': 'fp_4008e3b719'}

In [45]:
# Iterate through all the results
missing_count = 0
missing_list = list()
for i in range(len(image_paths)//3):
    try:
        print(i, ": ", results[i]['choices'][0]['message']['content'])
    except KeyError:
        print(i, ": missing")
        missing_count += 1
        missing_list.append(i)
print("\nmissing count: ", missing_count)
print("missing list: ", missing_list)
# for i in range(len(image_paths)):
#     print(i, ": ", results[i]['choices'][0]['message']['content'])

0 :  **Image Description:**
- Main Subjects: A person is standing on a wooden pier that extends into a calm lake.
- Actions: The person is standing still and facing away from the camera, seemingly admiring the view. They are wearing a red jacket and holding an object in their left hand.
- Context: The scene is set in a serene outdoor location, surrounded by a reflective lake and majestic mountains in the background. The sky is overcast, and the overall atmosphere is peaceful. The clear reflection of the mountains and trees in the still water suggests a tranquil and untouched natural environment.
1 :  **Image Description:**
- Main Subjects: A man and a woman are standing close together, smiling at the camera.
- Actions: The man is holding a selfie stick, taking a photo of them both.
- Context: They are on a viewing platform in front of a large, white waterfall cascading down a rocky cliff, indicating a scenic travel destination.

**Image Description:**
- Main Subjects: A map of North Am

In [48]:
embedding_model = 'text-embedding-3-small'
texts = results

def get_embedding(text, model=embedding_model):
    return client.embeddings.create(input=[text], model=model).data[0].embedding

def process_embeddings_in_parallel(results, model=embedding_model, num_threads=10):
    def embed_result(result):
        return get_embedding(result['choices'][0]['message']['content'], model=model)
    
    embedded_frame = {}

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_index = {executor.submit(embed_result, result): i for i, result in enumerate(results)}
        for future in concurrent.futures.as_completed(future_to_index):
            i = future_to_index[future]
            try:
                embedding = future.result()
                embedded_frame[i] = np.array(embedding).reshape(1, -1)
            except Exception as exc:
                print(f'Embedding for result {i} generated an exception: {exc}')

    return embedded_frame

# Usage
embedded_frame = process_embeddings_in_parallel(texts, model=embedding_model, num_threads=30)

# Embed the query
embedded_query = get_embedding(user_input, model=embedding_model)
embedded_query = np.array(embedded_query).reshape(1, -1)

print(embedded_frame)
print(embedded_query)

Embedding for result 67 generated an exception: 'choices'
{8: array([[ 0.01374399, -0.02203527, -0.01087901, ..., -0.01202104,
        -0.00228241,  0.01485301]]), 10: array([[ 0.03062559, -0.01621687, -0.02634535, ..., -0.02448069,
        -0.01353996,  0.02137293]]), 3: array([[-0.01219853, -0.01730375,  0.04808234, ..., -0.03060678,
        -0.01225989,  0.00321531]]), 16: array([[ 0.0140624 , -0.03923885, -0.04236383, ...,  0.00135614,
        -0.01940203, -0.00053328]]), 2: array([[ 0.01535222, -0.00903115, -0.05944591, ...,  0.00974895,
        -0.03937669,  0.00805699]]), 13: array([[ 0.02858688, -0.04091065, -0.01618326, ..., -0.01801985,
        -0.01401396,  0.01213079]]), 9: array([[ 0.04491668, -0.03057087, -0.00668655, ..., -0.01267831,
         0.0234112 ,  0.02137314]]), 18: array([[ 0.01617743, -0.03425809, -0.00039042, ...,  0.00186414,
        -0.02743366,  0.02353205]]), 12: array([[-0.00396864, -0.01089021, -0.00783956, ..., -0.04171789,
         0.01055513,  0.0009

In [ ]:
# embedding_model = 'text-embedding-3-small'
# def get_embedding(text, model=embedding_model):
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

# embedded_frame = {}
# for i in range(len(results)):
#     embedded_frame[i] = get_embedding(results[i]['choices'][0]['message']['content'], model=embedding_model)
#     embedded_frame[i] = np.array(embedded_frame[i]).reshape(1,-1)

# embedded_query = get_embedding(user_input, model=embedding_model)
# embedded_query = np.array(embedded_query).reshape(1,-1)

In [77]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_result = {}
for i in range(len(image_paths)//3):
    try:
        similarity_result[i] = cosine_similarity(embedded_frame[i], embedded_query)
    except KeyError:
        similarity_result[i] = 0
        
ranked_frame_dict = dict(sorted(similarity_result.items(), key=lambda x:x[1], reverse=True))

In [78]:
ranked_frame_list = list(ranked_frame_dict.keys())
ranked_frame_list[:10]

[230, 39, 91, 25, 44, 272, 347, 326, 233, 43]

In [57]:
for i in ranked_frame_list[:10]:
    print(results[i]['choices'][0]['message']['content'])

### Great Moments

#### Image 1
**Image Description:**
- Main Subjects: A grizzly bear.
- Actions: The grizzly bear is walking along the edge of a road.
- Context: The road is bordered by grass and wildflowers, with a dense forest of tall trees in the background. A sightings tally in the upper left corner lists sightings of various animals, with 1 grizzly bear noted.

#### Image 2
**Image Description:**
- Main Subjects: The same grizzly bear.
- Actions: The bear is in mid-stride, moving towards the camera's right.
- Context: The setting remains the same as in the first image, with the bear walking along a road bordered by wildflowers and a forest. This image is taken from a closer angle, offering a better view of the bear's profile.

#### Image 3
**Image Description:**
- Main Subjects: The same grizzly bear, with the rear of a car visible.
- Actions: The bear continues to walk along the roadside.
- Context: This image captures the scene from a slightly different angle, showing a car on